In [ ]:
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_style("whitegrid")

# magic number from efficiencies.csv; report TES storage and H2 storage in MWh electricity rather than heat
efficiency_csp_power_block = 0.41
efficiency_h2_ccgt = 0.58

# Select data to plot
df = pd.read_csv(snakemake.input["results"], delimiter=";")
df = df.loc[
    (df["exporter"] == snakemake.wildcards["exporter"])
    & (df["esc"] == snakemake.wildcards["esc"])
]
df = df.loc[df["category"].isin(["capacity factor", "installed capacity"])]

# Extract technology to aggreagate/group by later
df["technology"] = df["subcategory"].str.extract("^(.*)\s\d*|\(.*\)$")

# Calculate weighted CF for technology (rather than individual PyPSA components)
# Transformations and intermediary data storage
df = df.set_index(["scenario", "subcategory"])
data = df.query("category == 'installed capacity'")
data = data[["technology", "value"]]
data = data.rename(columns={"value": "capacity"})
data["capacity factor"] = df.query("category == 'capacity factor'")["value"]
data["CF*CAP"] = data["capacity factor"] * data["capacity"]

# this data will be used for plotting
plot_df = (
    data.groupby(["scenario", "technology"])["CF*CAP"].sum()
    / data.groupby(["scenario", "technology"])["capacity"].sum()
)
plot_df = plot_df.to_frame("weighted CF")
plot_df["total capacity"] = data.groupby(["scenario", "technology"])["capacity"].sum()
plot_df["total capacity"] /= 1e3  # in GWh
plot_df = plot_df.reset_index()

# CSP-TES and H2 storage in MWh elec instead of heat
plot_df.loc[
    plot_df["technology"] == "csp-tower TES", "total capacity"
] *= efficiency_csp_power_block
if "h2" in snakemake.wildcards["esc"]:
    plot_df.loc[
        plot_df["technology"] == "hydrogen storage tank type 1", "total capacity"
    ] *= efficiency_h2_ccgt

plot_df["flexibility"] = plot_df["scenario"].str.replace("_.*", "", regex=True)
plot_df["csp"] = plot_df["scenario"].str.contains("with_csp")

# Add sorted category column to df indicating type of flexibility
# then sort by that category to ensure correct order in x-axis of plot
# Note: Reproduces the order given in "flexibilities".

flexibilities = {
    "unbuffered": "No flexibility",
    "daily": "Daily",
    "weekly": "Weekly",
    "biweekly": "Biweekly",
    "monthly": "Month",
    "quaterly": "Quaterly",
    "annually": "Yearly",
}

plot_df["flexibility"] = plot_df.flexibility.replace(flexibilities)

plot_df["flexibility"] = pd.Categorical(
    plot_df.flexibility, categories=flexibilities.values(), ordered=True
)

plot_df = plot_df.sort_values("flexibility")

# Cleaning: Don't report CF and capacity for negligible capacities close to zero
plot_df.loc[plot_df["total capacity"] < 1, "weighted CF"] = 0
plot_df.loc[plot_df["total capacity"] < 1, "total capacity"] = 0

# Plot
fig, ax = plt.subplots(figsize=(8, 5))
ax2 = ax.twinx()

# Plot transmission infrastructure, depending on ESC
if "pipeline" in snakemake.wildcards["esc"]:
    tech = "H2 (g) fill compressor station"
    tech_label = "Pipeline"
else:
    tech = "HVDC inverter pair"
    tech_label = "HVDC"
color = "black"
csp = False
plot_df.query("technology == @tech and csp == @csp").plot(
    x="flexibility",
    y="weighted CF",
    ax=ax,
    marker="<",
    markerfacecolor=color if csp else "white",
    label=tech_label + " (w/   CSP, TES)" if csp else tech_label + " (w/o CSP, TES)",
    color=color,
    # linestyle="dashed"
)
csp = True
plot_df.query("technology == @tech and csp == @csp").plot(
    x="flexibility",
    y="weighted CF",
    ax=ax,
    marker="<",
    markerfacecolor=color if csp else "white",
    label=tech_label + " (w/   CSP, TES)" if csp else tech_label + " (w/o CSP, TES)",
    color=color,
    # linestyle="dashed"
)

csp = True
tech = "csp-tower TES"
tech_label = "CSP-TES"
color = "orange"
plot_df.query("technology == @tech and csp == @csp").plot(
    x="flexibility",
    y="total capacity",
    ax=ax2,
    marker=">",
    markerfacecolor=color if csp else "white",
    label=tech_label + " (w/   CSP, TES)" if csp else tech_label + " (w/o CSP, TES)",
    color=color,
    linestyle="dotted",
)

# Battery only shows relevant capacity for HVDC ESC,
# do not plot for h2 pipeline ESC case
if "hvdc" in snakemake.wildcards["esc"]:
    csp = False
    tech = "battery storage"
    tech_label = "Battery"
    color = "blue"
    plot_df.query("technology == @tech and csp == @csp").plot(
        x="flexibility",
        y="weighted CF",
        ax=ax,
        marker="<",
        markerfacecolor=color if csp else "white",
        label=tech_label + " (w/ CSP, TES)" if csp else tech_label + " (w/o CSP, TES)",
        color=color,
        # linestyle="dashed"
    )

    csp = True
    plot_df.query("technology == @tech and csp == @csp").plot(
        x="flexibility",
        y="weighted CF",
        ax=ax,
        marker="<",
        markerfacecolor=color if csp else "white",
        label=tech_label + " (w/   CSP, TES)"
        if csp
        else tech_label + " (w/o CSP, TES)",
        color=color,
        # linestyle="dashed"
    )

    csp = False
    plot_df.query("technology == @tech and csp == @csp").plot(
        x="flexibility",
        y="total capacity",
        ax=ax2,
        marker=">",
        markerfacecolor=color if csp else "white",
        label=tech_label + " (w/   CSP, TES)"
        if csp
        else tech_label + " (w/o CSP, TES)",
        color=color,
        linestyle="dotted",
    )

    csp = True
    plot_df.query("technology == @tech and csp == @csp").plot(
        x="flexibility",
        y="total capacity",
        ax=ax2,
        marker=">",
        markerfacecolor=color if csp else "white",
        label=tech_label + " (w/   CSP, TES)"
        if csp
        else tech_label + " (w/o CSP, TES)",
        color=color,
        linestyle="dotted",
    )

if "pipeline-h2" in snakemake.wildcards["esc"]:
    tech = "hydrogen storage tank type 1"
    tech_label = "Hydrogen storage"
    color = "green"

    csp = False
    plot_df.query("technology == @tech and csp == @csp").plot(
        x="flexibility",
        y="weighted CF",
        ax=ax,
        marker="<",
        markerfacecolor=color if csp else "white",
        label=tech_label + " (w/   CSP, TES)"
        if csp
        else tech_label + " (w/o CSP, TES)",
        color=color,
        # linestyle="dashed"
    )

    csp = True
    plot_df.query("technology == @tech and csp == @csp").plot(
        x="flexibility",
        y="weighted CF",
        ax=ax,
        marker="<",
        markerfacecolor=color if csp else "white",
        label=tech_label + " (w/   CSP, TES)"
        if csp
        else tech_label + " (w/o CSP, TES)",
        color=color,
        # linestyle="dashed"
    )

    csp = False
    plot_df.query("technology == @tech and csp == @csp").plot(
        x="flexibility",
        y="total capacity",
        ax=ax2,
        marker=">",
        markerfacecolor=color if csp else "white",
        label=tech_label + " (w/   CSP, TES)"
        if csp
        else tech_label + " (w/o CSP, TES)",
        color=color,
        linestyle="dotted",
    )

    csp = True
    plot_df.query("technology == @tech and csp == @csp").plot(
        x="flexibility",
        y="total capacity",
        ax=ax2,
        marker=">",
        markerfacecolor=color if csp else "white",
        label=tech_label + " (w/   CSP, TES)"
        if csp
        else tech_label + " (w/o CSP, TES)",
        color=color,
        linestyle="dotted",
    )

ax.set_xlabel("Flexibility to shift load")

ax.set_ylim(0, 1.05)
ax.set_ylabel("Utilisation factor [p.u.]")
ax.legend(bbox_to_anchor=(0, 1.0), title="Utilisation factor", loc="lower left")

ax2.set_ylim(0, 250 * 1.05)  # Magic values for consistency for all ESCs
ax2.set_yticks([0, 50, 100, 150, 200, 250])
ax2.set_ylabel("Capacity [$\mathregular{GWh}_{e}$]")
ax2.legend(bbox_to_anchor=(1.0, 1.0), title="Capacity", loc="lower right")

# Saving figure
fig.savefig(snakemake.output["pdf"], dpi=300, bbox_inches="tight")
fig.savefig(snakemake.output["png"], dpi=300, bbox_inches="tight")